In [1]:
# code borrowed from https://sdsawtelle.github.io/blog/output/boardgamegeek-data-scraping.html
# modified to more closely fit my needs

import requests
from bs4 import BeautifulSoup
import scipy.io
import matplotlib.pyplot as plt
import matplotlib 
import pandas as pd
import numpy as np
import pickle
from time import sleep
import timeit


In [2]:
def request(msg, slp=1):
    '''A wrapper to make robust https requests.'''
    while True:
        try:
            r = requests.get(msg)
            if r.status_code == 200:
                return r
            print("Server Error! Response Code %i. Retrying..." % (r.status_code))
            sleep(slp)
        except:
            print("An exception has occurred, probably a momentory loss of connection. Waiting one seconds...")
            sleep(1)


In [3]:
# Initialize a DF to hold all our scraped game info
df_all = pd.DataFrame(columns=["id", "name", "rating", "nrate"])
min_nrate = 501
npage = 1

# Scrap successful pages in the results until we get down to games with < 100 ratings each
while min_nrate > 100:
    # Get full HTML for a specific page in the full listing of boardgames sorted by 
    r = request("https://boardgamegeek.com/browse/boardgame/page/%i?sort=numvoters&sortdir=desc" % (npage,))
    soup = BeautifulSoup(r.text, "html.parser")    
    
    # Get rows for the table listing all the games on this page
    table = soup.find_all("tr", attrs={"id": "row_"})  # Get list of all the rows (tags) in the list of games on this page
    df = pd.DataFrame(columns=["id", "name", "rating", "nrate"], index=range(len(table)))  # DF to hold this pages results
    
    # Loop through each row and pull out the info for that game
    for idx, row in enumerate(table):
        # Row may or may not start with a "boardgame rank" link, if YES then strip it
        links = row.find_all("a")
        if "name" in links[0].attrs.keys():
            del links[0]
        gamelink = links[1]  # Get the relative URL for the specific game
        gameid = int(gamelink["href"].split("/")[2])  # Get the game ID by parsing the relative URL
        gamename = gamelink.contents[0]  # Get the actual name of the game as the link contents

        ratings = row.find_all("td", attrs={"class": "collection_bggrating"})
        avg = float("".join(ratings[1].contents[0].split()))
        nratings = int("".join(ratings[2].contents[0].split()))

        df.iloc[idx, :] = [gameid, gamename, avg, nratings]

    # Concatenate the results of this page to the master dataframe
    min_nrate = df["nrate"].min()  # The smallest number of ratings of any game on the page
    print("Page %i scraped, minimum number of ratings was %i" % (npage, min_nrate))
    df_all = pd.concat([df_all, df], axis=0, ignore_index=True)
    npage += 1
    sleep(2) # Keep the BGG server happy.

Page 1 scraped, minimum number of ratings was 16193
Page 2 scraped, minimum number of ratings was 10645
Page 3 scraped, minimum number of ratings was 7520
Page 4 scraped, minimum number of ratings was 5925
Page 5 scraped, minimum number of ratings was 4750
Page 6 scraped, minimum number of ratings was 4077
Page 7 scraped, minimum number of ratings was 3536
Page 8 scraped, minimum number of ratings was 3086
Page 9 scraped, minimum number of ratings was 2742
Page 10 scraped, minimum number of ratings was 2437
Page 11 scraped, minimum number of ratings was 2221
Page 12 scraped, minimum number of ratings was 2006
Page 13 scraped, minimum number of ratings was 1807
Page 14 scraped, minimum number of ratings was 1676
Page 15 scraped, minimum number of ratings was 1558
Page 16 scraped, minimum number of ratings was 1459
Page 17 scraped, minimum number of ratings was 1373
Page 18 scraped, minimum number of ratings was 1287
Page 19 scraped, minimum number of ratings was 1210
Page 20 scraped, mi

In [7]:
df_all['nrate'].sum()

12352214

In [6]:
# Prepare a "# of FULL pages of ratings" column to track # API calls needed
df_all["nfullpages"] = (df_all["nrate"]-50).apply(round, ndigits=-2)/100  # Round DOWN to nearest 100
df_all.head()

,id,name,rating,nrate,nfullpages
0,13,Catan,7.23,76523,765.0
1,822,Carcassonne,7.43,76119,761.0
2,30549,Pandemic,7.66,73223,732.0
3,36218,Dominion,7.67,62426,624.0
4,68448,7 Wonders,7.81,60610,606.0


In [8]:
import sqlite3
connex = sqlite3.connect("ratings.db")
cur = connex.cursor()

In [10]:
df_toy = df_all.loc[df_all["nrate"] < 5000, ].copy()


In [12]:
from IPython.display import clear_output


#############################################################
# Gathering all ratings from all games in toy data set
#############################################################
# Get ratings page-by-page for all games, but do it in chunks of 250 games
for nm, grp in df_toy.groupby(np.arange(len(df_toy))//10):
    if nm < 117:
        continue
    # Initialize a DF to hold all the responses for this chunk of games
    df_ratings = pd.DataFrame(columns=["gameid", "username", "rating"], index=range(grp["nrate"].sum()+100000))

    # Initialize indices for writing to the ratings dataframe
    dfidx_start = 0
    dfidx = 0
    
    # For this group of games, make calls until all FULL pages of every game have been pulled
    pagenum = 1
    while len(grp[grp["nfullpages"] > 0]) > 0:
        clear_output()
        print("%i: %i" % (nm, pagenum))
        # Get a restricted DF with only still-active games (have ratings pages left)
        active_games = grp[grp["nfullpages"] > 0]

        # Set the next chunk of the DF "gameid" column using the list of game IDs
        id_list = []
        for game in active_games["id"]:
            id_list += [game]*100
        dfidx_end = dfidx_start + len(active_games)*100
        df_ratings.iloc[dfidx_start:dfidx_end, df_ratings.columns.get_loc("gameid")] = id_list

        # Make the request with the list of all game IDs that have ratings left
        id_strs = [str(gid) for gid in active_games["id"]]
        gameids = ",".join(id_strs)
        url = "https://www.boardgamegeek.com/xmlapi2/thing?id=%s&ratingcomments=1&page=%i" % (gameids, pagenum)
        r = request(url, 20)
        soup = BeautifulSoup(r.text, "xml")
        comments = soup("comment")

        # Parse the response and assign it into the dataframe
        l1 = [0]*len(active_games)*100
        l2 = [0]*len(active_games)*100
        for j, comm in enumerate(comments):
            l1[j] = comm["username"]
            l2[j] = float(comm["rating"])
        df_ratings.iloc[dfidx_start:dfidx_end, df_ratings.columns.get_loc("username")] = l1
        df_ratings.iloc[dfidx_start:dfidx_end, df_ratings.columns.get_loc("rating")] = l2

        
        grp["nfullpages"] -= 1  # Decrement the number of FULL pages of each game id
        dfidx_start = dfidx_end     
        pagenum += 1
        sleep(10)  # Keep the server happy
    
    # Strip off the empty rows
    df_ratings = df_ratings.dropna(how="all")
    # Write this batch of all FULL pages of ratings for this chunk of games to the DB
    df_ratings.to_sql(name="data", con=connex, if_exists="append", index=False)    
    print("Processed ratings for batch #%i of games." % (nm))

1021: 1
Processed ratings for batch #1021 of games.
Processed ratings for batch #1022 of games.


In [121]:
print("Dropping the 20 most-rated games would give a total number of ratings of %i" 
      % (df.loc[500:, "nrate"].sum(),))

Dropping the 20 most-rated games would give a total number of ratings of 4731


In [85]:
df_ratings["rating"][68101]

4.0

In [13]:
#############################################################
# Request the final partial page of ratings for each game
#############################################################
# Restore the correct number of FULL pages
df_toy["nfullpages"] = (df_toy["nrate"]-50).apply(round, ndigits=-2)/100  # Round DOWN to nearest 100, then divide by 100

# Initialize a DF to hold all the responses over all the chunks of games
df_ratings = pd.DataFrame(columns=["gameid", "username", "rating"], index=range(len(df_toy)*100))

# Initialize indices for writing to the ratings dataframe
dfidx_start = 0
dfidx = 0

# Loop through game-by-game and request the final page of ratings for each game
for idx, row in df_toy.iterrows():
    # Get the game ID and the last page number to request
    pagenum = row["nfullpages"] + 1
    gameid = row["id"]
    
    # Make the request for just the last page of ratings of this game
    sleep(1)  # Keep the server happy
    r = request("http://www.boardgamegeek.com/xmlapi2/thing?id=%i&ratingcomments=1&page=%i" % (gameid, pagenum), 5)
    soup = BeautifulSoup(r.text, "xml")
    comments = soup("comment")
#         print("Response status was %i - length of comments is %i" % (r.status_code, len(comments)))

    # Set the next chunk of the DF "gameids" column with this gameid
    id_list = [gameid]*len(comments)
    dfidx_end = dfidx_start + len(comments)
    df_ratings.iloc[dfidx_start:dfidx_end, df_ratings.columns.get_loc("gameid")] = id_list

    # Parse the response and assign it into the dataframe
    l1 = [0]*len(comments)
    l2 = [0]*len(comments)
    j = 0
    for comm in comments:
        l1[j] = comm["username"]
        l2[j] = float(comm["rating"])
        j += 1
    df_ratings.iloc[dfidx_start:dfidx_end, df_ratings.columns.get_loc("username")] = l1
    df_ratings.iloc[dfidx_start:dfidx_end, df_ratings.columns.get_loc("rating")] = l2

    dfidx_start = dfidx_end   # Increment the starting index for next round        

    if idx%100 == 0:
        print("Finished with a chunk of 100 games.")
        
# Strip off the empty rows
df_ratings = df_ratings.dropna(how="all")

# Write this final batch of all partial pages of ratings for this chunk of games to the DB
df_ratings.to_sql(name="data", con=connex, if_exists="append", index=False)  

Finished with a chunk of 100 games.
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Finished with a chunk of 100 games.
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Finished with a chunk of 100

Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Finished with a chunk of 100 games.
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Respo

Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Finished with a chunk of 100 games.
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Respo

Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Finished with a chunk of 100 games.
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Respo

Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Finished with a chunk of 100 games.
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Respo

Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Finished with a chunk of 100 games.
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Respo

Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Finished with a chunk of 100 games.
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Finished with a chunk of 100 games.
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 

An exception has occurred, probably a momentory loss of connection. Waiting one seconds...
Finished with a chunk of 100 games.
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Finished with a chunk of 100 games.
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Finished with a chunk of 100 games.
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Finished with a chunk of 100 games.
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...
Server Error! Response Code 429. Retrying...


In [13]:
connex = sqlite3.connect("ratings.db")  # Opens file if exists, else creates file
cur = connex.cursor()  # This object lets us actually send messages to our DB and receive results
sql = "SELECT * FROM data" + ";"
df_ratings = pd.read_sql_query(sql, connex)
df_ratings.sample(n=10)

,gameid,username,rating
3524176,129293,srand,4.0
82144,624,Selebration,6.0
4865452,220502,SolanKaranga,9.0
3269046,10816,mtyree1972,5.0
4477907,10908,sedge,6.0
1353410,177,KalevTait,8.0
1958287,145196,CptWasp,8.7
2354591,195544,Ruskicowboy,7.0
5431285,308,shilinski,6.0
4727261,114871,BGDigger,6.0


In [14]:
# Check that our data matches server data
fig, axs = plt.subplots(figsize=[16, 4], nrows=1, ncols=3)
axs = axs.ravel()
for idx, game in enumerate(df_toy["id"].sample(n=3)):
    df = df_ratings[df_ratings["gameid"] == game]
    nm = df_toy.loc[df_toy["id"] == game, "name"].values
    __ = axs[idx].hist(df["rating"], bins=10, normed=True)
    axs[idx].set_title("%s (%i)" % (nm, game))
    print("%s Our Data: Mean = %.2f. StdDev = %.2f" % (nm, df["rating"].mean(), df["rating"].std()))
    
    # Request actual stats from the server to compare with scraped data
    r = requests.get("http://www.boardgamegeek.com/xmlapi2/thing?id=%i&stats=1" % (game,))
    soup = BeautifulSoup(r.text, "xml")
    std = float(soup("stddev")[0]["value"])
    mn = float(soup("average")[0]["value"])
    print("%s Server Stats: Mean = %.2f. StdDev = %.2f" % (nm, mn,std))
    sleep(1.5)

['Piratoons'] Our Data: Mean = 6.58. StdDev = 1.38
['Piratoons'] Server Stats: Mean = 6.58. StdDev = 1.38
['Agricola: All Creatures Big and Small – More Buildings Big and Small'] Our Data: Mean = 7.77. StdDev = 1.05
['Agricola: All Creatures Big and Small – More Buildings Big and Small'] Server Stats: Mean = 7.77. StdDev = 1.05
['Wooolf!!'] Our Data: Mean = 6.70. StdDev = 1.44
['Wooolf!!'] Server Stats: Mean = 6.70. StdDev = 1.44


In [25]:
# Only keep users with more than 10 ratings
# A nifty and somewhat fault-tolerant way of constructing long SQL queries
sql = " ".join((
    "SELECT username",
    "FROM (SELECT username, count(*) as freqn FROM data GROUP BY username)",
    "AS tbl WHERE freqn > 10",
))

users = pd.read_sql_query(sql, connex)
users.sample(n=3)

ProgrammingError: Cannot operate on a closed database.

In [15]:
# same as above
sql = "SELECT username FROM (SELECT username FROM data GROUP BY username) AS tbl"
users = pd.read_sql_query(sql, connex)
usrs = ["'" + usr + "'" for usr in users["username"].values]



In [16]:
len(usrs)

209771

In [22]:
df_ratings.to_csv("ratings.csv", index=False)

In [ ]:
connex.close()